### The Data
This book builds on previous notebooks regarding feature enginnering and feature selection however in this book we will train a model per 1JHC and 1JHN coupling types. We will additionally feature enhance those datasets by more details about the eletronic structure of the molecules.

Note: this book will only consider data from the following csv files: train, structures and molecule_structures


In [1]:
import pandas as pd

input_folder = './input'

train = pd.read_csv(f'{input_folder}/feature_files/1JHN.csv')


In [2]:
pd.set_option('display.max_columns', None)
    
train.head()

,atom_index_0,atom_index_1,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,atom_0_CH3,atom_1_CH3,atom_0_CH2,atom_1_CH2,atom_0_CH1,atom_1_CH1,atom_0_CH0,atom_1_CH0,atom_0_NH2,atom_1_NH2,atom_0_NH1,atom_1_NH1,atom_0_NH0,atom_1_NH0,atom_0_OH1,atom_1_OH1,atom_0_OH0,atom_1_OH0,is_in_ring,has_aromatic_bond,is_primary_amide,is_secondary_amide,is_tertiary_amide,is_ester,is_carbonyl,bond_order,NB_left_atom,NB_left_group,NB_center_atom,NB_center_group,NB_right_atom,NB_right_group,NB_all_groups,NB_in_ring,NB_is_aromatic,NB_is_primary_amide,NB_is_secondary_amide,NB_is_tertiary_amide,NB_is_ester,NB_is_carbonyl,NB_is_hydroxyl,abs_dx,abs_dy,abs_dz,distance,molecule_couples,molecule_dist_mean,molecule_dist_min,molecule_dist_max,molecule_dist_std,atom_0_couples_count,atom_1_couples_count,molecule_atom_index_0_count,molecule_atom_index_1_count,molecule_atom_index_0_distance_mean,molecule_atom_index_0_distance_mean_diff,molecule_atom_index_0_distance_mean_div,molecule_atom_index_0_distance_std,molecule_atom_index_0_distance_std_diff,molecule_atom_index_0_distance_std_div,molecule_atom_index_0_distance_max,molecule_atom_index_0_distance_max_diff,molecule_atom_index_0_distance_max_div,molecule_atom_index_0_distance_min,molecule_atom_index_0_distance_min_diff,molecule_atom_index_0_distance_min_div,molecule_atom_index_1_distance_mean,molecule_atom_index_1_distance_mean_diff,molecule_atom_index_1_distance_mean_div,molecule_atom_index_1_distance_std,molecule_atom_index_1_distance_std_diff,molecule_atom_index_1_distance_std_div,molecule_atom_index_1_distance_max,molecule_atom_index_1_distance_max_diff,molecule_atom_index_1_distance_max_div,molecule_atom_index_1_distance_min,molecule_atom_index_1_distance_min_diff,molecule_atom_index_1_distance_min_div
0,1,0,32.6889,H,0.017257,0.012545,-0.027377,N,-0.040426,1.024108,0.062564,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0.057684,1.011563,0.089941,1.017190,6,1.317921,1.017187,1.618710,0.329428,3,3,6,6,1.418141,0.400951,1.394175,0.347234,-0.669956,0.341366,1.618710,0.601520,1.591354,1.017190,0.0,1.0,1.017195,0.000005,1.000005,0.000011,-1.017179,0.000011,1.017208,0.000018,1.000018,1.017187,-0.000003,0.999997
1,2,0,32.6891,H,0.915789,1.358745,-0.028758,N,-0.040426,1.024108,0.062564,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0.956215,0.334637,0.091322,1.017187,6,1.317921,1.017187,1.618710,0.329428,2,3,6,6,1.317946,0.300759,1.295677,0.425338,-0.591849,0.418151,1.618706,0.601518,1.591355,1.017187,0.0,1.0,1.017195,0.000008,1.000008,0.000011,-1.017176,0.000011,1.017208,0.000021,1.000020,1.017187,0.000000,1.000000
2,3,0,32.6905,H,-0.520278,1.343532,-0.775543,N,-0.040426,1.024108,0.062564,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0.479852,0.319424,0.838106,1.017208,6,1.317921,1.017187,1.618710,0.329428,1,3,6,6,1.017208,0.000000,1.000000,NaN,NaN,NaN,1.017208,0.000000,1.000000,1.017208,0.0,1.0,1.017195,-0.000013,0.999987,0.000011,-1.017197,0.000011,1.017208,0.000000,1.000000,1.017187,-0.000021,0.999980
3,3,0,55.5252,H,0.825355,1.885049,0.003738,N,-0.025900,1.346146,0.008894,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,C,CH1,CH1,0,0,0,0,0,0,1,0,0.851255,0.538904,0.005156,1.007511,9,1.809863,1.004933,2.960154,0.664909,4,3,9,9,1.938232,0.930721,1.923782,0.809230,-0.198281,0.803197,2.960154,1.952643,2.938085,1.007511,0.0,1.0,1.353280,0.345768,1.343191,0.601122,-0.406389,0.596641,2.047394,1.039883,2.032131,1.004933,-0.002578,0.997441
4,4,0,54.7359,H,-0.908377,1.826796,0.018920,N,-0.025900,1.346146,0.008894,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,C,CH1,CH1,0,0,0,0,0,0,1,0,0.882477,0.480651,0.010026,1.004933,9,1.809863,1.004933,2.960154,0.664909,3,3,9,9,1.793050,0.788117,1.784248,0.692204,-0.312729,0.688806,2.302437,1.297504,2.291134,1.004933,0.0,1.0,1.353280,0.348346,1.346636,0.601122,-0.403811,0.598171

In [3]:
train.shape

(43363, 89)

In [4]:
import numpy as np

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                    df[col] = df[col].astype(np.float16)
                else:
                    df[col] = df[col].astype(np.float32)
                    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
train = reduce_mem_usage(train)
train.dtypes

Mem. usage decreased to 11.12 Mb (62.2% reduction)


atom_index_0                                   int8
atom_index_1                                   int8
scalar_coupling_constant                    float32
atom_0                                       object
x_0                                         float32
y_0                                         float32
z_0                                         float32
atom_1                                       object
x_1                                         float32
y_1                                         float32
z_1                                         float32
atom_0_CH3                                     int8
atom_1_CH3                                     int8
atom_0_CH2                                     int8
atom_1_CH2                                     int8
atom_0_CH1                                     int8
atom_1_CH1                                     int8
atom_0_CH0                                     int8
atom_1_CH0                                     int8
atom_0_NH2  

#### Remove none important features

In [6]:
train = train.drop('NB_left_atom', axis=1)
train = train.drop('NB_left_group', axis=1)

train = train.drop('NB_center_atom', axis=1)
train = train.drop('NB_center_group', axis=1)

train = train.drop('NB_right_atom', axis=1)
train = train.drop('NB_right_group', axis=1)

train = train.drop('x_0', axis=1)
train = train.drop('y_0', axis=1)
train = train.drop('z_0', axis=1)

train = train.drop('x_1', axis=1)
train = train.drop('y_1', axis=1)
train = train.drop('z_1', axis=1)

train = train.drop('molecule_atom_index_0_distance_std', axis=1)
train = train.drop('molecule_atom_index_1_distance_std', axis=1)

train = train.drop('molecule_atom_index_0_distance_min_div', axis=1)
train = train.drop('molecule_atom_index_1_distance_min_div', axis=1)

train = train.drop('molecule_atom_index_0_distance_min_diff', axis=1)
train = train.drop('molecule_atom_index_1_distance_min_diff', axis=1)

train = train.drop('molecule_atom_index_0_distance_max_div', axis=1)
train = train.drop('molecule_atom_index_1_distance_max_div', axis=1)

train = train.drop('molecule_atom_index_0_distance_max_diff', axis=1)
train = train.drop('molecule_atom_index_1_distance_max_diff', axis=1)

train = train.drop('molecule_atom_index_0_distance_std_div', axis=1)
train = train.drop('molecule_atom_index_1_distance_std_div', axis=1)

train = train.drop('molecule_atom_index_0_distance_std_diff', axis=1)
train = train.drop('molecule_atom_index_1_distance_std_diff', axis=1)

train = train.drop('molecule_atom_index_0_distance_mean_div', axis=1)
train = train.drop('molecule_atom_index_1_distance_mean_div', axis=1)

train = train.drop('molecule_atom_index_0_distance_mean_diff', axis=1)
train = train.drop('molecule_atom_index_1_distance_mean_diff', axis=1)



#### Fill NaN

In [8]:
train[train.isna().any(axis=1)]

,atom_index_0,atom_index_1,scalar_coupling_constant,atom_0,atom_1,atom_0_CH3,atom_1_CH3,atom_0_CH2,atom_1_CH2,atom_0_CH1,atom_1_CH1,atom_0_CH0,atom_1_CH0,atom_0_NH2,atom_1_NH2,atom_0_NH1,atom_1_NH1,atom_0_NH0,atom_1_NH0,atom_0_OH1,atom_1_OH1,atom_0_OH0,atom_1_OH0,is_in_ring,has_aromatic_bond,is_primary_amide,is_secondary_amide,is_tertiary_amide,is_ester,is_carbonyl,bond_order,NB_all_groups,NB_in_ring,NB_is_aromatic,NB_is_primary_amide,NB_is_secondary_amide,NB_is_tertiary_amide,NB_is_ester,NB_is_carbonyl,NB_is_hydroxyl,abs_dx,abs_dy,abs_dz,distance,molecule_couples,molecule_dist_mean,molecule_dist_min,molecule_dist_max,molecule_dist_std,atom_0_couples_count,atom_1_couples_count,molecule_atom_index_0_count,molecule_atom_index_1_count,molecule_atom_index_0_distance_mean,molecule_atom_index_0_distance_max,molecule_atom_index_0_distance_min,molecule_atom_index_1_distance_mean,molecule_atom_index_1_distance_max,molecule_atom_index_1_distance_min


In [7]:
# If needed based on outcome of above

train.NB_all_groups.fillna('', inplace=True)
train.fillna(0, inplace=True)

In [9]:
train.head()

,atom_index_0,atom_index_1,scalar_coupling_constant,atom_0,atom_1,atom_0_CH3,atom_1_CH3,atom_0_CH2,atom_1_CH2,atom_0_CH1,atom_1_CH1,atom_0_CH0,atom_1_CH0,atom_0_NH2,atom_1_NH2,atom_0_NH1,atom_1_NH1,atom_0_NH0,atom_1_NH0,atom_0_OH1,atom_1_OH1,atom_0_OH0,atom_1_OH0,is_in_ring,has_aromatic_bond,is_primary_amide,is_secondary_amide,is_tertiary_amide,is_ester,is_carbonyl,bond_order,NB_all_groups,NB_in_ring,NB_is_aromatic,NB_is_primary_amide,NB_is_secondary_amide,NB_is_tertiary_amide,NB_is_ester,NB_is_carbonyl,NB_is_hydroxyl,abs_dx,abs_dy,abs_dz,distance,molecule_couples,molecule_dist_mean,molecule_dist_min,molecule_dist_max,molecule_dist_std,atom_0_couples_count,atom_1_couples_count,molecule_atom_index_0_count,molecule_atom_index_1_count,molecule_atom_index_0_distance_mean,molecule_atom_index_0_distance_max,molecule_atom_index_0_distance_min,molecule_atom_index_1_distance_mean,molecule_atom_index_1_distance_max,molecule_atom_index_1_distance_min
0,1,0,32.688900,H,N,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,,0,0,0,0,0,0,0,0,0.057684,1.011563,0.089941,1.017190,6,1.317921,1.017187,1.618710,0.329428,3,3,6,6,1.418141,1.618710,1.017190,1.017195,1.017208,1.017187
1,2,0,32.689098,H,N,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,,0,0,0,0,0,0,0,0,0.956215,0.334637,0.091322,1.017187,6,1.317921,1.017187,1.618710,0.329428,2,3,6,6,1.317946,1.618706,1.017187,1.017195,1.017208,1.017187
2,3,0,32.690498,H,N,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,,0,0,0,0,0,0,0,0,0.479852,0.319424,0.838106,1.017208,6,1.317921,1.017187,1.618710,0.329428,1,3,6,6,1.017208,1.017208,1.017208,1.017195,1.017208,1.017187
3,3,0,55.525200,H,N,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,CH1,0,0,0,0,0,0,1,0,0.851255,0.538904,0.005156,1.007511,9,1.809863,1.004933,2.960154,0.664909,4,3,9,9,1.938232,2.960154,1.007511,1.353280,2.047394,1.004933
4,4,0,54.735901,H,N,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,CH1,0,0,0,0,0,0,1,0,0.882477,0.480651,0.010026,1.004933,9,1.809863,1.004933,2.960154,0.664909,3,3,9,9,1.793050,2.302437,1.004933,1.353280,2.047394,1.004933


### Data splitting

In [10]:
from sklearn.model_selection import train_test_split

y = train['scalar_coupling_constant']
X = train.drop('scalar_coupling_constant', axis=1)
    
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 13, test_size = 0.05)

#### Cleanup - we need the resources

In [11]:
import gc

del train

gc.collect()

30

### The model

In [12]:
import numpy as np
from catboost import CatBoostRegressor

cat_features_index = np.where(X_train.dtypes != np.float32)[0]

model = CatBoostRegressor(iterations=5000, depth= 14, random_seed = 23, task_type = "GPU", devices='0', gpu_ram_part=0.95, gpu_cat_features_storage='CpuPinnedMemory')
model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=cat_features_index, logging_level='Verbose')

# test: 2.1113 1JHC {iter=2000, depth=13, all features}
# test: 1.5779 1JHN {iter=2000, depth=13, all features}
# test: 1.5171 1JHN {iter=2000, depth=13, only NB_all_groups}
# test: 1.4704 1JHN {iter=2500, depth=14, only NB_all_groups}



0:	learn: 47.2578898	test: 47.4801504	best: 47.4801504 (0)	total: 291ms	remaining: 24m 12s
1:	learn: 45.8551688	test: 46.0704529	best: 46.0704529 (1)	total: 1.3s	remaining: 53m 59s
2:	learn: 44.4934112	test: 44.7063107	best: 44.7063107 (2)	total: 3.42s	remaining: 1h 34m 54s
3:	learn: 43.1715135	test: 43.3797097	best: 43.3797097 (3)	total: 4.14s	remaining: 1h 26m 10s
4:	learn: 41.8911121	test: 42.0940953	best: 42.0940953 (4)	total: 6.33s	remaining: 1h 45m 20s
5:	learn: 40.6444230	test: 40.8433725	best: 40.8433725 (5)	total: 8.36s	remaining: 1h 55m 56s
6:	learn: 39.4388825	test: 39.6356446	best: 39.6356446 (6)	total: 10.4s	remaining: 2h 3m 47s
7:	learn: 38.2688849	test: 38.4602400	best: 38.4602400 (7)	total: 10.7s	remaining: 1h 50m 57s
8:	learn: 37.1359646	test: 37.3260295	best: 37.3260295 (8)	total: 12.6s	remaining: 1h 56m 32s
9:	learn: 36.0336394	test: 36.2194069	best: 36.2194069 (9)	total: 13.2s	remaining: 1h 50m 2s
10:	learn: 34.9673177	test: 35.1479502	best: 35.1479502 (10)	total: 1

87:	learn: 4.1005963	test: 4.0825253	best: 4.0825253 (87)	total: 1m 59s	remaining: 1h 51m 34s
88:	learn: 4.0131192	test: 3.9923560	best: 3.9923560 (88)	total: 2m	remaining: 1h 50m 59s
89:	learn: 3.9272779	test: 3.9038746	best: 3.9038746 (89)	total: 2m 2s	remaining: 1h 51m 24s
90:	learn: 3.8452780	test: 3.8204554	best: 3.8204554 (90)	total: 2m 2s	remaining: 1h 50m 29s
91:	learn: 3.7653708	test: 3.7387048	best: 3.7387048 (91)	total: 2m 5s	remaining: 1h 51m 14s
92:	learn: 3.6916801	test: 3.6624270	best: 3.6624270 (92)	total: 2m 7s	remaining: 1h 51m 54s
93:	learn: 3.6193944	test: 3.5885005	best: 3.5885005 (93)	total: 2m 9s	remaining: 1h 52m 33s
94:	learn: 3.5491808	test: 3.5165751	best: 3.5165751 (94)	total: 2m 10s	remaining: 1h 51m 54s
95:	learn: 3.4824411	test: 3.4471706	best: 3.4471706 (95)	total: 2m 11s	remaining: 1h 52m 18s
96:	learn: 3.4148608	test: 3.3794177	best: 3.3794177 (96)	total: 2m 13s	remaining: 1h 52m 40s
97:	learn: 3.3538696	test: 3.3171798	best: 3.3171798 (97)	total: 2m 1

174:	learn: 1.8717250	test: 1.7853494	best: 1.7853494 (174)	total: 4m 41s	remaining: 2h 9m 13s
175:	learn: 1.8700307	test: 1.7831635	best: 1.7831635 (175)	total: 4m 43s	remaining: 2h 9m 24s
176:	learn: 1.8687006	test: 1.7816990	best: 1.7816990 (176)	total: 4m 43s	remaining: 2h 8m 49s
177:	learn: 1.8676357	test: 1.7804742	best: 1.7804742 (177)	total: 4m 45s	remaining: 2h 9m
178:	learn: 1.8665358	test: 1.7792167	best: 1.7792167 (178)	total: 4m 46s	remaining: 2h 8m 29s
179:	learn: 1.8570668	test: 1.7711068	best: 1.7711068 (179)	total: 4m 48s	remaining: 2h 8m 42s
180:	learn: 1.8558594	test: 1.7697910	best: 1.7697910 (180)	total: 4m 48s	remaining: 2h 8m 13s
181:	learn: 1.8494116	test: 1.7637930	best: 1.7637930 (181)	total: 4m 51s	remaining: 2h 8m 24s
182:	learn: 1.8486429	test: 1.7628825	best: 1.7628825 (182)	total: 4m 52s	remaining: 2h 8m 31s
183:	learn: 1.8475064	test: 1.7618237	best: 1.7618237 (183)	total: 4m 54s	remaining: 2h 8m 37s
184:	learn: 1.8464827	test: 1.7608216	best: 1.7608216 

262:	learn: 1.7874362	test: 1.7064625	best: 1.7064625 (262)	total: 6m 25s	remaining: 1h 55m 34s
263:	learn: 1.7874250	test: 1.7064322	best: 1.7064322 (263)	total: 6m 25s	remaining: 1h 55m 9s
264:	learn: 1.7874033	test: 1.7064528	best: 1.7064322 (263)	total: 6m 26s	remaining: 1h 54m 57s
265:	learn: 1.7873871	test: 1.7064243	best: 1.7064243 (265)	total: 6m 26s	remaining: 1h 54m 37s
266:	learn: 1.7873744	test: 1.7063947	best: 1.7063947 (266)	total: 6m 26s	remaining: 1h 54m 15s
267:	learn: 1.7873546	test: 1.7064135	best: 1.7063947 (266)	total: 6m 28s	remaining: 1h 54m 16s
268:	learn: 1.7873408	test: 1.7063910	best: 1.7063910 (268)	total: 6m 29s	remaining: 1h 54m 5s
269:	learn: 1.7873238	test: 1.7064136	best: 1.7063910 (268)	total: 6m 29s	remaining: 1h 53m 42s
270:	learn: 1.7873168	test: 1.7063931	best: 1.7063910 (268)	total: 6m 29s	remaining: 1h 53m 18s
271:	learn: 1.7812866	test: 1.7002595	best: 1.7002595 (271)	total: 6m 31s	remaining: 1h 53m 29s
272:	learn: 1.7749623	test: 1.6947337	best

348:	learn: 1.6822739	test: 1.6163561	best: 1.6163561 (348)	total: 7m 48s	remaining: 1h 44m 5s
349:	learn: 1.6754906	test: 1.6107268	best: 1.6107268 (349)	total: 7m 50s	remaining: 1h 44m 14s
350:	learn: 1.6728197	test: 1.6073232	best: 1.6073232 (350)	total: 7m 52s	remaining: 1h 44m 22s
351:	learn: 1.6696081	test: 1.6047720	best: 1.6047720 (351)	total: 7m 54s	remaining: 1h 44m 30s
352:	learn: 1.6696050	test: 1.6047888	best: 1.6047720 (351)	total: 7m 55s	remaining: 1h 44m 15s
353:	learn: 1.6682794	test: 1.6036193	best: 1.6036193 (353)	total: 7m 57s	remaining: 1h 44m 23s
354:	learn: 1.6682792	test: 1.6036169	best: 1.6036169 (354)	total: 7m 57s	remaining: 1h 44m 5s
355:	learn: 1.6682776	test: 1.6036139	best: 1.6036139 (355)	total: 7m 57s	remaining: 1h 43m 49s
356:	learn: 1.6678299	test: 1.6032190	best: 1.6032190 (356)	total: 7m 59s	remaining: 1h 43m 55s
357:	learn: 1.6677474	test: 1.6032049	best: 1.6032049 (357)	total: 8m 1s	remaining: 1h 43m 59s
358:	learn: 1.6677471	test: 1.6032026	best:

...

4600:	learn: 1.4181968	test: 1.4214943	best: 1.4214943 (4592)	total: 32m 36s	remaining: 2m 49s
4601:	learn: 1.4181967	test: 1.4214941	best: 1.4214941 (4601)	total: 32m 37s	remaining: 2m 49s
4602:	learn: 1.4181968	test: 1.4214941	best: 1.4214941 (4601)	total: 32m 37s	remaining: 2m 48s
4603:	learn: 1.4181968	test: 1.4214941	best: 1.4214941 (4603)	total: 32m 37s	remaining: 2m 48s
4604:	learn: 1.4181967	test: 1.4214940	best: 1.4214940 (4604)	total: 32m 37s	remaining: 2m 47s
4605:	learn: 1.4181967	test: 1.4214939	best: 1.4214939 (4605)	total: 32m 37s	remaining: 2m 47s
4606:	learn: 1.4181967	test: 1.4214937	best: 1.4214937 (4606)	total: 32m 37s	remaining: 2m 47s
4607:	learn: 1.4181966	test: 1.4214936	best: 1.4214936 (4607)	total: 32m 38s	remaining: 2m 46s
4608:	learn: 1.4181966	test: 1.4214934	best: 1.4214934 (4608)	total: 32m 38s	remaining: 2m 46s
4609:	learn: 1.4181966	test: 1.4214933	best: 1.4214933 (4609)	total: 32m 38s	remaining: 2m 45s
4610:	learn: 1.4181966	test: 1.4214932	best: 1.421

4688:	learn: 1.4172308	test: 1.4210091	best: 1.4210089 (4675)	total: 32m 59s	remaining: 2m 11s
4689:	learn: 1.4172308	test: 1.4210091	best: 1.4210089 (4675)	total: 32m 59s	remaining: 2m 10s
4690:	learn: 1.4172308	test: 1.4210091	best: 1.4210089 (4675)	total: 32m 59s	remaining: 2m 10s
4691:	learn: 1.4172308	test: 1.4210091	best: 1.4210089 (4675)	total: 32m 59s	remaining: 2m 9s
4692:	learn: 1.4172308	test: 1.4210091	best: 1.4210089 (4675)	total: 33m	remaining: 2m 9s
4693:	learn: 1.4172308	test: 1.4210091	best: 1.4210089 (4675)	total: 33m	remaining: 2m 9s
4694:	learn: 1.4172308	test: 1.4210090	best: 1.4210089 (4675)	total: 33m	remaining: 2m 8s
4695:	learn: 1.4172308	test: 1.4210090	best: 1.4210089 (4675)	total: 33m	remaining: 2m 8s
4696:	learn: 1.4172308	test: 1.4210090	best: 1.4210089 (4675)	total: 33m	remaining: 2m 7s
4697:	learn: 1.4172308	test: 1.4210090	best: 1.4210089 (4675)	total: 33m	remaining: 2m 7s
4698:	learn: 1.4172308	test: 1.4210090	best: 1.4210089 (4675)	total: 33m	remainin

4776:	learn: 1.4151455	test: 1.4196937	best: 1.4196912 (4770)	total: 33m 23s	remaining: 1m 33s
4777:	learn: 1.4151342	test: 1.4196882	best: 1.4196882 (4777)	total: 33m 23s	remaining: 1m 33s
4778:	learn: 1.4151342	test: 1.4196882	best: 1.4196882 (4777)	total: 33m 24s	remaining: 1m 32s
4779:	learn: 1.4151342	test: 1.4196882	best: 1.4196882 (4777)	total: 33m 24s	remaining: 1m 32s
4780:	learn: 1.4151342	test: 1.4196882	best: 1.4196882 (4777)	total: 33m 24s	remaining: 1m 31s
4781:	learn: 1.4150930	test: 1.4196610	best: 1.4196610 (4781)	total: 33m 26s	remaining: 1m 31s
4782:	learn: 1.4150917	test: 1.4196534	best: 1.4196534 (4782)	total: 33m 26s	remaining: 1m 31s
4783:	learn: 1.4150916	test: 1.4196546	best: 1.4196534 (4782)	total: 33m 26s	remaining: 1m 30s
4784:	learn: 1.4150902	test: 1.4196472	best: 1.4196472 (4784)	total: 33m 26s	remaining: 1m 30s
4785:	learn: 1.4150902	test: 1.4196472	best: 1.4196472 (4785)	total: 33m 27s	remaining: 1m 29s
4786:	learn: 1.4150889	test: 1.4196410	best: 1.419

4863:	learn: 1.4125077	test: 1.4174916	best: 1.4174892 (4826)	total: 33m 47s	remaining: 56.7s
4864:	learn: 1.4125077	test: 1.4174916	best: 1.4174892 (4826)	total: 33m 47s	remaining: 56.3s
4865:	learn: 1.4125076	test: 1.4174916	best: 1.4174892 (4826)	total: 33m 47s	remaining: 55.8s
4866:	learn: 1.4125077	test: 1.4174916	best: 1.4174892 (4826)	total: 33m 47s	remaining: 55.4s
4867:	learn: 1.4125077	test: 1.4174916	best: 1.4174892 (4826)	total: 33m 47s	remaining: 55s
4868:	learn: 1.4125077	test: 1.4174925	best: 1.4174892 (4826)	total: 33m 47s	remaining: 54.6s
4869:	learn: 1.4125077	test: 1.4174925	best: 1.4174892 (4826)	total: 33m 48s	remaining: 54.1s
4870:	learn: 1.4125077	test: 1.4174925	best: 1.4174892 (4826)	total: 33m 48s	remaining: 53.7s
4871:	learn: 1.4125077	test: 1.4174924	best: 1.4174892 (4826)	total: 33m 48s	remaining: 53.3s
4872:	learn: 1.4125077	test: 1.4174934	best: 1.4174892 (4826)	total: 33m 48s	remaining: 52.9s
4873:	learn: 1.4125077	test: 1.4174944	best: 1.4174892 (4826)	

4952:	learn: 1.4114118	test: 1.4163765	best: 1.4163765 (4951)	total: 34m 5s	remaining: 19.4s
4953:	learn: 1.4114118	test: 1.4163764	best: 1.4163764 (4953)	total: 34m 5s	remaining: 19s
4954:	learn: 1.4114118	test: 1.4163750	best: 1.4163750 (4954)	total: 34m 6s	remaining: 18.6s
4955:	learn: 1.4114095	test: 1.4163780	best: 1.4163750 (4954)	total: 34m 6s	remaining: 18.2s
4956:	learn: 1.4114095	test: 1.4163766	best: 1.4163750 (4954)	total: 34m 6s	remaining: 17.8s
4957:	learn: 1.4114095	test: 1.4163766	best: 1.4163750 (4954)	total: 34m 6s	remaining: 17.3s
4958:	learn: 1.4114095	test: 1.4163752	best: 1.4163750 (4954)	total: 34m 6s	remaining: 16.9s
4959:	learn: 1.4114095	test: 1.4163752	best: 1.4163750 (4954)	total: 34m 6s	remaining: 16.5s
4960:	learn: 1.4114095	test: 1.4163752	best: 1.4163750 (4954)	total: 34m 7s	remaining: 16.1s
4961:	learn: 1.4114095	test: 1.4163739	best: 1.4163739 (4961)	total: 34m 7s	remaining: 15.7s
4962:	learn: 1.4114094	test: 1.4163738	best: 1.4163738 (4962)	total: 34m

#### Helper functions to compare with:

In [13]:
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

def show_scores(y_valid,y_pred):
    print("Variance_Score(cat_Regressor)\t:"+str(explained_variance_score(y_valid,y_pred)))
    print("Mean_Absolute_Error(cat_Regressor)\t:"+str(mean_absolute_error(y_valid,y_pred)))
    print("Mean_Squared_Error(cat_Regressor)\t:"+str(mean_squared_error(y_valid,y_pred)))
    print("R2-Score(cat_Regressor)\t:"+str(r2_score(y_valid,y_pred)))
    
def show_feature_importance(model, columns):
    fea_imp = pd.DataFrame({'col': columns, 'imp': model.feature_importances_})
    fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-50:]
    fea_imp.plot(kind='barh', x='col', y='imp', figsize=(10, 7), legend=None)
    plt.title('CatBoost - Feature Importance')
    plt.ylabel('Features')
    plt.xlabel('Importance')

#### 1JHN

In [14]:
y_pred = model.predict(X_valid)

In [15]:
show_scores(y_valid, y_pred)

Variance_Score(cat_Regressor)	:0.9834649912968269
Mean_Absolute_Error(cat_Regressor)	:1.028266205690708
Mean_Squared_Error(cat_Regressor)	:2.003045969530408
R2-Score(cat_Regressor)	:0.9834436838457461


In [ ]:
# no one-hot encoding 2000 iter, D=14 - has molecule structue features + has additional feature enhancements - 
# Only used NB-all_groups removed x_0, y_0.. distances and several more exotic aggregated features
Variance_Score(cat_Regressor)	:0.9820377715317836
Mean_Absolute_Error(cat_Regressor)	:1.0427346216941829
Mean_Squared_Error(cat_Regressor)	:2.162078766001682
R2-Score(cat_Regressor)	:0.9820265056331184

In [ ]:
# no one-hot encoding 2000 iter, D=13 - has molecule structue features + has additional feature enhancements - 
# Only used NB-all_groups, removed molecule_atom_index_0_distance_std, molecule_atom_index_0_distance_std_diff, molecule_atom_index_0_distance_std_div and equivalent for atom_index_1

# Note: bestIteration = 1999 --> still declining
Variance_Score(cat_Regressor)	:0.9808776732226606
Mean_Absolute_Error(cat_Regressor)	:1.0872139573401178
Mean_Squared_Error(cat_Regressor)	:2.301625015776636
R2-Score(cat_Regressor)	:0.980866449036805

In [ ]:
# no one-hot encoding 2000 iter, D=13 - has molecule structue features + has additional feature enhancements - all features
Variance_Score(cat_Regressor)	:0.9793132895640081
Mean_Absolute_Error(cat_Regressor)	:1.141982051830773
Mean_Squared_Error(cat_Regressor)	:2.4898595327215554
R2-Score(cat_Regressor)	:0.9793016438672788

In [ ]:
# no one-hot encoding 2000 iter, D=13 - has molecule structue features + has additional feature enhancements
Variance_Score(cat_Regressor)	:0.9750208042502584
Mean_Absolute_Error(cat_Regressor)	:1.2027936896520528
Mean_Squared_Error(cat_Regressor)	:3.0050552888841975
R2-Score(cat_Regressor)	:0.9750187897146733

In [ ]:
# no one-hot encoding 2000 iter, D=13 - only molecule structue features - no additional feature enhancements
Variance_Score(cat_Regressor)	:0.9298085539454239
Mean_Absolute_Error(cat_Regressor)	:2.0511599010813986
Mean_Squared_Error(cat_Regressor)	:8.445888514171434
R2-Score(cat_Regressor)	:0.9297888069482795

In [ ]:
# no one-hot encoding 2000 iter, D=13 - only additional feature enhancements - no molecule structue features
Variance_Score(cat_Regressor)	:0.9611473589018391
Mean_Absolute_Error(cat_Regressor)	:1.5373073117979494
Mean_Squared_Error(cat_Regressor)	:4.673770404443461
R2-Score(cat_Regressor)	:0.9611466578566376

In [ ]:
show_feature_importance(model, X_valid.columns)

### Let's save the model for later

In [ ]:
model.save_model('./input/models/model_1JHN', format="cbm")

#from catboost import CatBoostClassifier
#model = CatBoostClassifier()      # parameters not required.
#model.load_model('model_name')


### Adding predictions to submission file

In [16]:
import pandas as pd

input_folder = './input'

test = pd.read_csv(f'{input_folder}/test.csv')
structures = pd.read_csv(f'{input_folder}/structures.csv')
molecular_structures = pd.read_csv(f'{input_folder}/molecular_structures_with_NB_test.csv')

In [17]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

In [18]:
test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

In [19]:
def map_molecule_info(df):
    df = pd.merge(df, molecular_structures, how = 'left',
                  left_on  = ['molecule_name', 'atom_index_0', 'atom_index_1'],
                  right_on = ['molecule_name',  'atom_0', 'atom_1'])
    
    df = df.drop('atom_0_y', axis=1)
    df = df.drop('atom_1_y', axis=1)
    df = df.drop('atom_1_1_level_NB', axis=1)
    df = df.drop('atom_1_2_level_NB', axis=1)
    
    df = df.rename(columns={'atom_0_x': 'atom_0',
                            'atom_1_x': 'atom_1'})
    return df

In [20]:
test = map_molecule_info(test)

In [21]:
test.loc[test['molecule_name'] == 'dsgdb9nsd_000015']

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,atom_symbol_0,atom_symbol_1,atom_0_CH3,atom_1_CH3,atom_0_CH2,atom_1_CH2,atom_0_CH1,atom_1_CH1,atom_0_CH0,atom_1_CH0,atom_0_NH2,atom_1_NH2,atom_0_NH1,atom_1_NH1,atom_0_NH0,atom_1_NH0,atom_0_OH1,atom_1_OH1,atom_0_OH0,atom_1_OH0,is_in_ring,has_aromatic_bond,is_primary_amide,is_secondary_amide,is_tertiary_amide,is_ester,is_carbonyl,bond_order,bond_length,NB_left_atom,NB_left_group,NB_center_atom,NB_center_group,NB_right_atom,NB_right_group,NB_all_groups,NB_in_ring,NB_is_aromatic,NB_is_primary_amide,NB_is_secondary_amide,NB_is_tertiary_amide,NB_is_ester,NB_is_carbonyl,NB_is_hydroxyl
5,4658152,dsgdb9nsd_000015,3,0,1JHC,H,1.005284,1.810158,0.004656,C,-0.014821,1.392412,0.005671,H,C,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.102328,NaN,NaN,NaN,NaN,O,OH0,OH0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4658153,dsgdb9nsd_000015,3,2,3JHC,H,1.005284,1.810158,0.004656,C,0.637949,-0.553297,-1.113582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4658154,dsgdb9nsd_000015,3,4,2JHH,H,1.005284,1.810158,0.004656,H,-0.546896,1.793435,-0.872511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4658155,dsgdb9nsd_000015,3,5,2JHH,H,1.005284,1.810158,0.004656,H,-0.530029,1.722920,0.911017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4658156,dsgdb9nsd_000015,4,0,1JHC,H,-0.546896,1.793435,-0.872511,C,-0.014821,1.392412,0.005671,H,C,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.102327,NaN,NaN,NaN,NaN,O,OH0,OH0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,4658157,dsgdb9nsd_000015,4,2,3JHC,H,-0.546896,1.793435,-0.872511,C,0.637949,-0.553297,-1.113582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,4658158,dsgdb9nsd_000015,4,5,2JHH,H,-0.546896,1.793435,-0.872511,H,-0.530029,1.722920,0.911017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,4658159,dsgdb9nsd_000015,5,0,1JHC,H,-0.530029,1.722920,0.911017,C,-0.014821,1.392412,0.005671,H,C,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.092852,NaN,NaN,NaN,NaN,O,OH0,OH0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,4658160,dsgdb9nsd_000015,5,2,3JHC,H,-0.530029,1.722920,0.911017,C,0.637949,-0.553297,-1.113582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,4658161,dsgdb9nsd_000015,6,0,3JHC,H,0.139938,-0.255993,-2.050984,C,-0.014821,1.392412,0.005671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
import numpy as np

test.iloc[:,2:4] = test.iloc[:,2:4].astype(np.int8)
test.iloc[:,13:15] = test.iloc[:,13:15].fillna('')
test.iloc[:,15:41] = test.iloc[:,15:41].fillna(0.0).astype(np.int8)
test.iloc[:,42:49] = test.iloc[:,42:49].fillna('')
test.iloc[:,49:57] = test.iloc[:,49:57].fillna(0.0).astype(np.int8)
test.iloc[:,41:42] = test.iloc[:,41:42].fillna(1)

In [23]:
test.dtypes

id                         int64
molecule_name             object
atom_index_0                int8
atom_index_1                int8
type                      object
atom_0                    object
x_0                      float64
y_0                      float64
z_0                      float64
atom_1                    object
x_1                      float64
y_1                      float64
z_1                      float64
atom_symbol_0             object
atom_symbol_1             object
atom_0_CH3                  int8
atom_1_CH3                  int8
atom_0_CH2                  int8
atom_1_CH2                  int8
atom_0_CH1                  int8
atom_1_CH1                  int8
atom_0_CH0                  int8
atom_1_CH0                  int8
atom_0_NH2                  int8
atom_1_NH2                  int8
atom_0_NH1                  int8
atom_1_NH1                  int8
atom_0_NH0                  int8
atom_1_NH0                  int8
atom_0_OH1                  int8
atom_1_OH1

In [24]:
test = test.drop('atom_symbol_0', axis=1)
test = test.drop('atom_symbol_1', axis=1)
# remove bond lenght as we will calculate our own
test = test.drop('bond_length', axis=1)

In [25]:
def add_distances(df):
    df['abs_dx']=(df['x_1']-df['x_0']).abs()
    df['abs_dy']=(df['y_1']-df['y_0']).abs()
    df['abs_dz']=(df['z_1']-df['z_0']).abs()
    df['distance']=(df['abs_dx']**2 + df['abs_dy']**2 + df['abs_dz']**2)**(1/2)
    return df

In [26]:
test=add_distances(test)

In [27]:
def additional_features(df):
    df['molecule_couples'] = df.groupby('molecule_name')['id'].transform('count')
    df['molecule_dist_mean'] = df.groupby('molecule_name')['distance'].transform('mean')
    df['molecule_dist_min'] = df.groupby('molecule_name')['distance'].transform('min')
    df['molecule_dist_max'] = df.groupby('molecule_name')['distance'].transform('max')
    df['molecule_dist_std'] = df.groupby('molecule_name')['distance'].transform('std')
    df['atom_0_couples_count'] = df.groupby(['molecule_name', 'atom_index_0'])['id'].transform('count')
    df['atom_1_couples_count'] = df.groupby(['molecule_name', 'atom_index_1'])['id'].transform('count')

    num_cols = ['distance'] # 'dx', 'dy', 'dz'
    cat_cols = ['atom_index_0', 'atom_index_1']
    aggs = ['mean', 'std', 'max', 'min']
    for col in cat_cols:
        df[f'molecule_{col}_count'] = df.groupby('molecule_name')[col].transform('count')

    for cat_col in cat_cols:
        for num_col in num_cols:
            for agg in aggs:
                df[f'molecule_{cat_col}_{num_col}_{agg}'] = df.groupby(['molecule_name', cat_col])[num_col].transform(agg)
                df[f'molecule_{cat_col}_{num_col}_{agg}_diff'] = df[f'molecule_{cat_col}_{num_col}_{agg}'] - df[num_col]
                df[f'molecule_{cat_col}_{num_col}_{agg}_div'] = df[f'molecule_{cat_col}_{num_col}_{agg}'] / df[num_col]

    df = reduce_mem_usage(df)
    return df

In [28]:
test=additional_features(test)

Mem. usage decreased to 707.28 Mb (38.8% reduction)


In [29]:
test.NB_all_groups.fillna('', inplace=True)
test.fillna(0, inplace=True)

In [30]:
test[test.isna().any(axis=1)]

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,atom_0_CH3,atom_1_CH3,atom_0_CH2,atom_1_CH2,atom_0_CH1,atom_1_CH1,atom_0_CH0,atom_1_CH0,atom_0_NH2,atom_1_NH2,atom_0_NH1,atom_1_NH1,atom_0_NH0,atom_1_NH0,atom_0_OH1,atom_1_OH1,atom_0_OH0,atom_1_OH0,is_in_ring,has_aromatic_bond,is_primary_amide,is_secondary_amide,is_tertiary_amide,is_ester,is_carbonyl,bond_order,NB_left_atom,NB_left_group,NB_center_atom,NB_center_group,NB_right_atom,NB_right_group,NB_all_groups,NB_in_ring,NB_is_aromatic,NB_is_primary_amide,NB_is_secondary_amide,NB_is_tertiary_amide,NB_is_ester,NB_is_carbonyl,NB_is_hydroxyl,abs_dx,abs_dy,abs_dz,distance,molecule_couples,molecule_dist_mean,molecule_dist_min,molecule_dist_max,molecule_dist_std,atom_0_couples_count,atom_1_couples_count,molecule_atom_index_0_count,molecule_atom_index_1_count,molecule_atom_index_0_distance_mean,molecule_atom_index_0_distance_mean_diff,molecule_atom_index_0_distance_mean_div,molecule_atom_index_0_distance_std,molecule_atom_index_0_distance_std_diff,molecule_atom_index_0_distance_std_div,molecule_atom_index_0_distance_max,molecule_atom_index_0_distance_max_diff,molecule_atom_index_0_distance_max_div,molecule_atom_index_0_distance_min,molecule_atom_index_0_distance_min_diff,molecule_atom_index_0_distance_min_div,molecule_atom_index_1_distance_mean,molecule_atom_index_1_distance_mean_diff,molecule_atom_index_1_distance_mean_div,molecule_atom_index_1_distance_std,molecule_atom_index_1_distance_std_diff,molecule_atom_index_1_distance_std_div,molecule_atom_index_1_distance_max,molecule_atom_index_1_distance_max_diff,molecule_atom_index_1_distance_max_div,molecule_atom_index_1_distance_min,molecule_atom_index_1_distance_min_diff,molecule_atom_index_1_distance_min_div


In [32]:
test.loc[test['molecule_name'] == 'dsgdb9nsd_000015']

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,atom_0_CH3,atom_1_CH3,atom_0_CH2,atom_1_CH2,atom_0_CH1,atom_1_CH1,atom_0_CH0,atom_1_CH0,atom_0_NH2,atom_1_NH2,atom_0_NH1,atom_1_NH1,atom_0_NH0,atom_1_NH0,atom_0_OH1,atom_1_OH1,atom_0_OH0,atom_1_OH0,is_in_ring,has_aromatic_bond,is_primary_amide,is_secondary_amide,is_tertiary_amide,is_ester,is_carbonyl,bond_order,NB_left_atom,NB_left_group,NB_center_atom,NB_center_group,NB_right_atom,NB_right_group,NB_all_groups,NB_in_ring,NB_is_aromatic,NB_is_primary_amide,NB_is_secondary_amide,NB_is_tertiary_amide,NB_is_ester,NB_is_carbonyl,NB_is_hydroxyl,abs_dx,abs_dy,abs_dz,distance,molecule_couples,molecule_dist_mean,molecule_dist_min,molecule_dist_max,molecule_dist_std,atom_0_couples_count,atom_1_couples_count,molecule_atom_index_0_count,molecule_atom_index_1_count,molecule_atom_index_0_distance_mean,molecule_atom_index_0_distance_mean_diff,molecule_atom_index_0_distance_mean_div,molecule_atom_index_0_distance_std,molecule_atom_index_0_distance_std_diff,molecule_atom_index_0_distance_std_div,molecule_atom_index_0_distance_max,molecule_atom_index_0_distance_max_diff,molecule_atom_index_0_distance_max_div,molecule_atom_index_0_distance_min,molecule_atom_index_0_distance_min_diff,molecule_atom_index_0_distance_min_div,molecule_atom_index_1_distance_mean,molecule_atom_index_1_distance_mean_diff,molecule_atom_index_1_distance_mean_div,molecule_atom_index_1_distance_std,molecule_atom_index_1_distance_std_diff,molecule_atom_index_1_distance_std_div,molecule_atom_index_1_distance_max,molecule_atom_index_1_distance_max_diff,molecule_atom_index_1_distance_max_div,molecule_atom_index_1_distance_min,molecule_atom_index_1_distance_min_diff,molecule_atom_index_1_distance_min_div
5,4658152,dsgdb9nsd_000015,3,0,1JHC,H,1.005284,1.810158,0.004656,C,-0.014821,1.392412,0.005671,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,,,,,O,OH0,OH0,0,0,0,0,0,0,0,0,1.020105,0.417746,0.001015,1.102328,18,1.910365,1.092852,3.262567,0.760446,4,6,18,18,1.827658,0.725330,1.657999,0.629906,-0.472422,0.571433,2.640324,1.537996,2.395226,1.102328,0.000000,1.000000,1.973384,0.871056,1.790197,0.984263,-0.118065,0.892895,3.262567,2.160239,2.959706,1.092852,-0.009476,0.991404
6,4658153,dsgdb9nsd_000015,3,2,3JHC,H,1.005284,1.810158,0.004656,C,0.637949,-0.553297,-1.113582,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,,,0,0,0,0,0,0,0,0,0.367335,2.363455,1.118239,2.640324,18,1.910365,1.092852,3.262567,0.760446,4,6,18,18,1.827658,-0.812666,0.692210,0.629906,-2.010418,0.238572,2.640324,0.000000,1.000000,1.102328,-1.537996,0.417497,1.973384,-0.666940,0.747402,0.984263,-1.656061,0.372781,3.262566,0.622242,1.235669,1.092852,-1.547472,0.413908
7,4658154,dsgdb9nsd_000015,3,4,2JHH,H,1.005284,1.810158,0.004656,H,-0.546896,1.793435,-0.872510,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,,,0,0,0,0,0,0,0,0,1.552179,0.016723,0.877167,1.782964,18,1.910365,1.092852,3.262567,0.760446,4,1,18,18,1.827658,0.044694,1.025067,0.629906,-1.153058,0.353292,2.640324,0.857360,1.480862,1.102328,-0.680636,0.618256,1.782964,0.000000,1.000000,0.000000,0.000000,0.000000,1.782964,0.000000,1.000000,1.782964,0.000000,1.000000
8,4658155,dsgdb9nsd_000015,3,5,2JHH,H,1.005284,1.810158,0.004656,H,-0.530029,1.722920,0.911017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,,,0,0,0,0,0,0,0,0,1.535313,0.087238,0.906361,1.785017,18,1.910365,1.092852,3.262567,0.760446,4,2,18,18,1.827658,0.042641,1.023889,0.629906,-1.155110,0.352885,2.640324,0.855307,1.479159,1.102328,-0.682689,0.617545,1.785009,-0.000008,0.999995,0.000011,-1.785005,0.000006,1.785017,0.000000,1.000000,1.785001,-0.000016,0.999991
9,4658156,dsgdb9nsd_000015,4,0,1JHC,H,-0.546896,1.793435,-0.872510,C,-0.014821,1.392412,0.005671,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,,,,,O,OH0,OH0,0,0,0,0,0,0,0,0,0.532074,0.401023,0.878182,1.102327,18,1.910365,1.092852,3.262567,0.760446,3,6,18,18,1.842412,0.740085,1.671384,0.770397,-0.331930,0.698883,2.63

In [33]:
test = test.drop('molecule_name', axis=1)
test = test.drop('id', axis=1)

In [34]:
import gc

del structures
del molecular_structures

gc.collect()

117

In [35]:
submission = pd.read_csv('./results/final/many_features_split_type_CAT_1JHC.csv')

In [36]:
test = test.drop('NB_left_atom', axis=1)
test = test.drop('NB_left_group', axis=1)

test = test.drop('NB_center_atom', axis=1)
test = test.drop('NB_center_group', axis=1)

test = test.drop('NB_right_atom', axis=1)
test = test.drop('NB_right_group', axis=1)

test = test.drop('x_0', axis=1)
test = test.drop('y_0', axis=1)
test = test.drop('z_0', axis=1)

test = test.drop('x_1', axis=1)
test = test.drop('y_1', axis=1)
test = test.drop('z_1', axis=1)

test = test.drop('molecule_atom_index_0_distance_std', axis=1)
test = test.drop('molecule_atom_index_1_distance_std', axis=1)

test = test.drop('molecule_atom_index_0_distance_min_div', axis=1)
test = test.drop('molecule_atom_index_1_distance_min_div', axis=1)

test = test.drop('molecule_atom_index_0_distance_min_diff', axis=1)
test = test.drop('molecule_atom_index_1_distance_min_diff', axis=1)

test = test.drop('molecule_atom_index_0_distance_max_div', axis=1)
test = test.drop('molecule_atom_index_1_distance_max_div', axis=1)

test = test.drop('molecule_atom_index_0_distance_max_diff', axis=1)
test = test.drop('molecule_atom_index_1_distance_max_diff', axis=1)

test = test.drop('molecule_atom_index_0_distance_std_div', axis=1)
test = test.drop('molecule_atom_index_1_distance_std_div', axis=1)

test = test.drop('molecule_atom_index_0_distance_std_diff', axis=1)
test = test.drop('molecule_atom_index_1_distance_std_diff', axis=1)

test = test.drop('molecule_atom_index_0_distance_mean_div', axis=1)
test = test.drop('molecule_atom_index_1_distance_mean_div', axis=1)

test = test.drop('molecule_atom_index_0_distance_mean_diff', axis=1)
test = test.drop('molecule_atom_index_1_distance_mean_diff', axis=1)

In [37]:
def score_data(index, row):
    row = row.drop(labels=['type'])
    row = row.tolist()
    y_pred = model.predict(row)
    #submission.at[index, 'scalar_coupling_constant'] = y_pred
    #print(submission.at[index, 'scalar_coupling_constant'])
    print(f"Old: {submission.at[index, 'scalar_coupling_constant']}; Pred: {y_pred}")


In [38]:
counter = 0
for index in range(test.shape[0]):
    counter += 1
    if counter > 500:
        break
    if index % 10000 == 0:
        print("Index: " + str(index))     
    
    row = test.iloc[index, :]
    if row.type in '1JHN':
        score_data(index, row)

Index: 0
Old: 52.524759455436325; Pred: 52.1106730727124
Old: 52.922413475916656; Pred: 52.91806856921822
Old: 53.24679043039218; Pred: 51.75847633623822
Old: 54.07751255442466; Pred: 53.035019094222534
Old: 58.29192269519904; Pred: 56.48273018099904
Old: 57.08934832860962; Pred: 58.09495447618893
Old: 57.603935129146564; Pred: 57.210461124973435


In [39]:
def score_data(index, row):
    row = row.drop(labels=['type'])
    row = row.tolist()
    y_pred = model.predict(row)
    submission.at[index, 'scalar_coupling_constant'] = y_pred

In [40]:
for index in range(test.shape[0]):
    if index % 10000 == 0:
        print("Index: " + str(index))     
    
    row = test.iloc[index, :]
    if row.type in '1JHN':
        score_data(index, row)

Index: 0
Index: 10000
Index: 20000
Index: 30000
Index: 40000
Index: 50000
Index: 60000
Index: 70000
Index: 80000
Index: 90000
Index: 100000
Index: 110000
Index: 120000
Index: 130000
Index: 140000
Index: 150000
Index: 160000
Index: 170000
Index: 180000
Index: 190000
Index: 200000
Index: 210000
Index: 220000
Index: 230000
Index: 240000
Index: 250000
Index: 260000
Index: 270000
Index: 280000
Index: 290000
Index: 300000
Index: 310000
Index: 320000
Index: 330000
Index: 340000
Index: 350000
Index: 360000
Index: 370000
Index: 380000
Index: 390000
Index: 400000
Index: 410000
Index: 420000
Index: 430000
Index: 440000
Index: 450000
Index: 460000
Index: 470000
Index: 480000
Index: 490000
Index: 500000
Index: 510000
Index: 520000
Index: 530000
Index: 540000
Index: 550000
Index: 560000
Index: 570000
Index: 580000
Index: 590000
Index: 600000
Index: 610000
Index: 620000
Index: 630000
Index: 640000
Index: 650000
Index: 660000
Index: 670000
Index: 680000
Index: 690000
Index: 700000
Index: 710000
Index:

In [41]:
submission.to_csv('./results/final/many_features_split_type_CAT_1JHC_1JHN.csv', index=False)